## analyse the study and results

In [4]:
import pickle as pkl
from typing import Any, Dict
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import math
import json
import datetime
import matplotlib.pyplot as plt 
%matplotlib inline
sys.path.append(os.path.abspath('./..'))
# from env_custom import CartPoleButter
import gym
import torch
import torch.nn as nn
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

with open("./study.pkl", "rb") as f:
    study = pkl.load(f)

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)
fig2.write_image('params.pdf')
fig1.write_image('hist.pdf')
fig1.show()
fig2.show()

In [4]:
df=study.trials_dataframe()
df.head()


,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_buffer_size,params_exploration_final_eps,params_exploration_fraction,params_gamma,params_learning_starts,params_lr,params_net_arch,params_target_update_interval,state
0,0,1.157441,2021-06-16 14:16:10.566945,2021-06-16 14:19:24.640900,0 days 00:03:14.073955,128,500000,0.180607,0.410026,0.900,20000,0.000712,small,10000,COMPLETE
1,1,869.523987,2021-06-16 14:16:10.568813,2021-06-16 14:24:08.134821,0 days 00:07:57.566008,512,10000,0.126771,0.466628,0.950,5000,0.000010,medium,5000,COMPLETE
2,2,849.635925,2021-06-16 14:16:10.570261,2021-06-16 14:22:15.694056,0 days 00:06:05.123795,1024,100000,0.006837,0.308446,0.995,10000,0.000086,small,1000,COMPLETE
3,3,1436.008423,2021-06-16 14:19:24.644020,2021-06-16 14:25:37.557056,0 days 00:06:12.913036,512,500000,0.109459,0.222944,0.999,20000,0.001186,medium,1000,COMPLETE
4,4,-404.595215,2021-06-16 14:22:15.695535,2021-06-16 14:27:09.104776,0 days 00:04:53.409241,128,10000,0.083656,0.499856,0.990,0,0.023900,small,1500,COMPLETE


In [5]:
df.sort_values(by=['value'],ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_buffer_size,params_exploration_final_eps,params_exploration_fraction,params_gamma,params_learning_starts,params_lr,params_net_arch,params_target_update_interval,state
308,308,1451.902832,2021-06-16 19:36:11.276090,2021-06-16 19:42:40.298867,0 days 00:06:29.022777,1024,100000,0.105190,0.197054,0.990,0,0.001421,small,200,COMPLETE
106,106,1442.917847,2021-06-16 16:21:29.935667,2021-06-16 16:27:56.342373,0 days 00:06:26.406706,1024,50000,0.111302,0.217195,0.999,0,0.001110,small,1500,COMPLETE
101,101,1441.085449,2021-06-16 16:13:09.038103,2021-06-16 16:20:08.522434,0 days 00:06:59.484331,1024,50000,0.104773,0.221781,0.999,0,0.001075,small,1500,COMPLETE
90,90,1439.819214,2021-06-16 16:04:42.822245,2021-06-16 16:10:26.498367,0 days 00:05:43.676122,1024,50000,0.086996,0.446224,0.990,5000,0.000298,small,1000,COMPLETE
120,120,1438.738281,2021-06-16 16:38:08.007436,2021-06-16 16:44:30.100001,0 days 00:06:22.092565,1024,50000,0.099204,0.228023,0.990,0,0.000282,small,1500,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,889,-407.843811,2021-06-17 03:39:06.044008,2021-06-17 03:39:11.999949,0 days 00:00:05.955941,1024,100000,0.064426,0.131955,0.999,20000,0.000479,small,200,PRUNED
523,523,-408.115662,2021-06-16 22:49:37.768055,2021-06-16 22:49:44.161788,0 days 00:00:06.393733,128,50000,0.127675,0.216110,0.999,20000,0.000463,small,1500,PRUNED
815,815,-408.730865,2021-06-17 02:39:26.390120,2021-06-17 02:39:30.980465,0 days 00:00:04.590345,1024,50000,0.094724,0.239130,0.990,20000,0.000129,medium,10000,PRUNED
481,481,-410.118011,2021-06-16 22:12:38.066113,2021-06-16 22:12:45.326479,0 days 00:00:07.260366,1024,10000,0.109885,0.209489,0.999,20000,0.000318,small,1000,PRUNED


In [6]:
print(df.loc[df['value'].argmax()])

number                                                  308
value                                           1451.902832
datetime_start                   2021-06-16 19:36:11.276090
datetime_complete                2021-06-16 19:42:40.298867
duration                             0 days 00:06:29.022777
params_batch_size                                      1024
params_buffer_size                                   100000
params_exploration_final_eps                       0.105190
params_exploration_fraction                        0.197054
params_gamma                                       0.990000
params_learning_starts                                    0
params_lr                                          0.001421
params_net_arch                                       small
params_target_update_interval                           200
state                                              COMPLETE
Name: 308, dtype: object
